In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from tqdm import tqdm

import pyroomacoustics as pra
import itertools

from pathlib import Path

import shamans.utils.eval_utils as eval

Used the data of experiment 3, but use threshold to find the number of sources.

In [12]:
# merge all the csv fo the experiments 1 for different runs
def merge_csvs_and_get_dataframe(exp_id, path_to_results):
    all_filenames = [i for i in path_to_results.glob(f'*_exp-{exp_id}_run-*.csv')]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv(path_to_results / f"experiment_results_exp-{exp_id}_all_runs.csv", index=False, encoding='utf-8-sig')
    return combined_csv

def merge_pickles_and_get_dict(exp_id, path_to_results):
    all_filenames = [i for i in path_to_results.glob(f'*_exp-{exp_id}_run-*_with_ang_specs.pkl')]
    combined_dicts = []
    for filename in tqdm(all_filenames):
        with open(filename, 'rb') as f:
            combined_dicts += pickle.load(f)
    with open(path_to_results / f"experiment_results_exp-{exp_id}_all_runs_with_ang_specs.pkl", 'wb') as f:
        pickle.dump(combined_dicts, f)
    return combined_dicts

In [13]:
# load data
exp_id = 3
path_to_results = Path('results/')
exp_df = merge_csvs_and_get_dataframe(exp_id, path_to_results)
ang_specs = merge_pickles_and_get_dict(exp_id, path_to_results)

100%|██████████| 4/4 [00:00<00:00, 79.99it/s]


In [14]:
exp_df['loc_method_simple'] = exp_df['loc_method'].apply(lambda x: x.split('-')[0])

best_seed = 666
# rename ref_8 and alg_6 to ref and alg
exp_df = exp_df.assign(sv_model=exp_df['sv_method'] + '_' + exp_df['nObs'].astype(str))
exp_df = exp_df.assign(loc_algo=exp_df['loc_method'].str[:3])
# exp_df['sv_model'] = exp_df['sv_model'].str.replace('ref_8', 'ref')
# exp_df['sv_model'] = exp_df['sv_model'].str.replace('alg_8', 'alg')
# exp_df.loc[exp_df['sv_model'] == 'ref', 'seed'] = 666
# exp_df.loc[exp_df['sv_model'] == 'alg', 'seed'] = 666

exp_df = exp_df[
    (exp_df['n_sources'] == 3)
    & (exp_df['loc_method_simple'].isin(['srp_phat', 'music_s', 'alpha']))
    & (exp_df['seed'] == 666)
    & (exp_df['duration'] == 1)
    & (exp_df['rt60'] > -1)
]
print(len(exp_df))

# laod picke file with the ground truth
with open(path_to_results / 'experiment_results_exp-3_all_runs_with_ang_specs.pkl', 'rb') as f:
    list_of_ang_specs = pickle.load(f)
    
print(exp_df.shape)
print(len(list_of_ang_specs))

doa_grid_rad = np.deg2rad(np.arange(0, 360, 6))

1620
(1620, 32)
1836


In [15]:
unique_frame_ids = exp_df['frame_id'].unique()
print(len(unique_frame_ids))
unique_method_ids = exp_df['method_id'].unique()
print(len(unique_method_ids))

12
45


In [16]:
print(unique_method_ids)

['music_s-1_freqs-[200, 4000]_gp-steerer_nObs-8_seed-666_norm-True'
 'music_s-1_freqs-[200, 4000]_gp-steerer_nObs-16_seed-666_norm-True'
 'music_s-1_freqs-[200, 4000]_gp-steerer_nObs-32_seed-666_norm-True'
 'music_s-1_freqs-[200, 4000]_gp-steerer_nObs-64_seed-666_norm-True'
 'music_s-1_freqs-[200, 4000]_gp-steerer_nObs-128_seed-666_norm-True'
 'music_s-4_freqs-[200, 4000]_gp-steerer_nObs-8_seed-666_norm-True'
 'music_s-4_freqs-[200, 4000]_gp-steerer_nObs-16_seed-666_norm-True'
 'music_s-4_freqs-[200, 4000]_gp-steerer_nObs-32_seed-666_norm-True'
 'music_s-4_freqs-[200, 4000]_gp-steerer_nObs-64_seed-666_norm-True'
 'music_s-4_freqs-[200, 4000]_gp-steerer_nObs-128_seed-666_norm-True'
 'music_s-3_freqs-[200, 4000]_gp-steerer_nObs-8_seed-666_norm-True'
 'music_s-3_freqs-[200, 4000]_gp-steerer_nObs-16_seed-666_norm-True'
 'music_s-3_freqs-[200, 4000]_gp-steerer_nObs-32_seed-666_norm-True'
 'music_s-3_freqs-[200, 4000]_gp-steerer_nObs-64_seed-666_norm-True'
 'music_s-3_freqs-[200, 4000]_gp-st

In [17]:
# initialize the dict with the frame ids
ang_specs_dicts = {}
for frame_id in unique_frame_ids:
    ang_specs_dicts[frame_id] = {}
    for method_id in unique_method_ids:
        ang_specs_dicts[frame_id][method_id] = None

for d in tqdm(list_of_ang_specs):
    if not d['frame_id'] in unique_frame_ids:
        continue
    if not d['method_id'] in unique_method_ids:
        continue
    # add the method id to the frame id
    ang_specs_dicts[d['frame_id']][d['method_id']] = d
    
# check if all the frames are there
missing_frames = []
retrieve_frames = []
for frame_id in unique_frame_ids:
    for method_id in unique_method_ids:
        if ang_specs_dicts[frame_id][method_id] is None:
            missing_frames.append((frame_id))
        else:
            retrieve_frames.append((frame_id))
            
print(len(np.unique(missing_frames)))
print(len(np.unique(retrieve_frames)))
print(np.unique(missing_frames))
print(np.unique(retrieve_frames))

100%|██████████| 1836/1836 [00:00<00:00, 123411.31it/s]

0
12
[]
['nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-1'
 'nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-1'
 'nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-1'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-0'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-0'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-0'
 'nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-2'
 'nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-2'
 'nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-2'
 'nSrc-3_doas-[49 16  9]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-3'
 'nSrc-3_doas-[49 16  9]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-3'
 'nSrc-3_doas-[49 16  9]_type-speech-durati

In [19]:
assert len(ang_specs_dicts.keys()) == len(unique_frame_ids), f"{len(ang_specs_dicts.keys())} != {len(unique_frame_ids)}"

In [20]:
def detect_peaks(values):
    # make circular
    n_points = len(values)
    val_ext = np.append(values, values[:10])

    # run peak finding
    indexes = pra.doa.detect_peaks(val_ext, show=False) % n_points
    candidates = np.unique(indexes)  # get rid of duplicates, if any
    return candidates

In [21]:
metrics_angular_threshold = 10
thresholds_ang_spec = [0., 0.05, 0.1, 0.2, 0.5, 0.6, 0.8, 0.9]

df_results = pd.DataFrame()

for thr in thresholds_ang_spec:
    
    for frame_id in tqdm(unique_frame_ids, desc='Iterating over frames for thr={}'.format(thr)):
        
        for method_id in unique_method_ids:
            
            ang_spec = ang_specs_dicts[frame_id][method_id]['ang_spec']
            
            ang_spec = np.mean(np.array(ang_spec), -1)
            ang_spec = ang_spec / np.max(ang_spec)
            
            # set the threshold
            ang_spec[ang_spec < thr] = 0
            
            # find peaks
            peaks = detect_peaks(ang_spec)     
            
            df_ = pd.DataFrame({
                'frame_id': frame_id,
                "method_id": method_id,
                'thresholds_ang_spec': thr,
                'peaks_locations': peaks.tolist(),
                'peaks_ids' : [f'p{i}' for i in range(len(peaks))],
                'n_peaks': len(peaks)
            })
            df_results = pd.concat([df_results, df_], ignore_index=True)
        
print(len(df_results))

Iterating over frames for thr=0.9: 100%|██████████| 12/12 [00:00<00:00, 45.82it/s]

18305


In [22]:
# merge the results_df with the exp_df on the frame_id and method_id
df_merge = exp_df.merge(df_results, on=['frame_id', 'method_id'])
print(len(df_merge))

54915


In [23]:
# for each frame id and method id, get the assorciate data_frame
metrics_angular_threshold = 15

results_thr_list = []

for frame_id in tqdm(unique_frame_ids, desc='Iterating over frames'):
    for method_id in unique_method_ids:
        
        df_ = df_merge.loc[
            (df_merge['frame_id'] == frame_id) 
            & (df_merge['method_id'] == method_id)
        ]
        
        thrs = df_['thresholds_ang_spec'].unique()
        
        for thr in thrs:
            
            n_sources = df_.loc[df_['thresholds_ang_spec'] == thr, 'n_sources'].unique()
            
            assert len(n_sources) == 1
            n_sources = n_sources[0]
            
            estimated_peaks_loc = df_.loc[df_['thresholds_ang_spec'] == thr, 'peaks_locations'].unique()
            estimated_azimuths = np.rad2deg(doa_grid_rad[estimated_peaks_loc])
            target_azimuths = np.rad2deg(df_.loc[df_['thresholds_ang_spec'] == thr, 'doas_ref_az'].unique())
            
            n_estimated = len(estimated_azimuths)
            n_target = len(target_azimuths)
            assert n_target == n_sources, f"{n_target} != {n_sources} \n {frame_id}"
            
            metrics = eval.compute_metrics(estimated_azimuths, target_azimuths, metrics_angular_threshold, np.rad2deg(doa_grid_rad))
            
            # check that tpr and fpr are not nans
            if np.isnan(metrics['tpr']) or np.isnan(metrics['fpr']):
                print(frame_id)
                print(estimated_azimuths)
                print(target_azimuths)
                print(n_sources)
                print(method_id)
                print(df_.loc[df_['thresholds_ang_spec'] == thr])
                raise ValueError('TPR or FPR is nan')
            
            
            if len(target_azimuths) > n_sources:
                print(frame_id)
                print(estimated_azimuths)
                print(target_azimuths)
                print(n_sources)
                print(method_id)
                print(df_.loc[df_['thresholds_ang_spec'] == thr])
                raise ValueError('True positives greater than n_sources')
            
            metrics['frame_id'] = frame_id
            metrics['method_id'] = method_id
            metrics['metrics_angular_threshold'] = metrics_angular_threshold
            metrics['thresholds_ang_spec'] = float(thr)
            
            results_thr_list.append(metrics)
        
df_results_thr_ = pd.DataFrame(results_thr_list)

Iterating over frames: 100%|██████████| 12/12 [00:05<00:00,  2.13it/s]


In [24]:
print(df_merge['frame_id'].unique())
print(df_results_thr_['frame_id'].unique())

['nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-2'
 'nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-2'
 'nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-2'
 'nSrc-3_doas-[49 16  9]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-3'
 'nSrc-3_doas-[49 16  9]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-3'
 'nSrc-3_doas-[49 16  9]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-3'
 'nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-1'
 'nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-1'
 'nSrc-3_doas-[ 2 51  3]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-1'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.0_mc-0'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-0'
 'nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-s

In [25]:
df_results_thr = df_results_thr_.merge(df_merge, on=['frame_id', 'method_id', 'thresholds_ang_spec'])
df_results_thr[:100].to_csv('asd.csv', index=False)
len(df_results_thr)

print(len(df_merge['method_id'].unique()))
print(len(df_results_thr_['method_id'].unique()))
print(len(df_results_thr['method_id'].unique()))

45
45
45


In [26]:
df_.columns

Index(['exp_name', 'time', 'record_id', 'num_srcs', 'src_ids', 'doas_est_idx',
       'doas_ref_idx', 'doas_ref_az', 'doas_est_az', 'doas_ref_el',
       'doas_est_el', 'errors', 'speech_files', 'frame_id', 'target_doa',
       'n_sources', 'duration', 'snr', 'noise_type', 'rt60', 'mc_seed',
       'method_id', 'loc_method', 'freq_min', 'freq_max', 'sv_method', 'nObs',
       'seed', 'sv_normalization', 'loc_method_simple', 'sv_model', 'loc_algo',
       'thresholds_ang_spec', 'peaks_locations', 'peaks_ids', 'n_peaks'],
      dtype='object')

In [27]:
print(method_id)
print(frame_id)
df_results_thr.loc[
    (df_results_thr['method_id'] == method_id)
    & (df_results_thr['frame_id'] == frame_id)
]

alpha-1.2_beta-0_eps-1E-3_iter-500_freqs-[200, 4000]_gp-steerer_nObs-128_seed-666_norm-True
nSrc-3_doas-[42 40 45]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.273_mc-0


,true_positives,false_positives,false_negatives,precision,recall,tpr,fpr,f1_score,accuracy,mean_error,...,sv_method,nObs,seed,sv_normalization,loc_method_simple,sv_model,loc_algo,peaks_locations,peaks_ids,n_peaks
54804,2,8,1,0.2,0.666667,0.666667,0.137931,0.307692,0.181818,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,2,p0,10
54805,2,8,1,0.2,0.666667,0.666667,0.137931,0.307692,0.181818,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,6,p1,10
54806,2,8,1,0.2,0.666667,0.666667,0.137931,0.307692,0.181818,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,18,p2,10
54807,2,8,1,0.2,0.666667,0.666667,0.137931,0.307692,0.181818,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,22,p3,10
54808,2,8,1,0.2,0.666667,0.666667,0.137931,0.307692,0.181818,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,27,p4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54910,1,0,2,1.0,0.333333,0.333333,0.000000,0.500000,0.333333,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,42,p0,1
54911,1,0,2,1.0,0.333333,0.333333,0.000000,0.500000,0.333333,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,42,p0,1
54912,1,0,2,1.0,0.333333,0.333333,0.000000,0.500000,0.333333,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,42,p0,1
54913,1,0,2,1.0,0.333333,0.333333,0.000000,0.500000,0.333333,0.0,...,gp-steerer,128,666,True,alpha,gp-steerer_128,alp,42,p0,1


In [ ]:
plt.figure(figsize=(8, 6))

thrs = np.sort(df_['thresholds_ang_spec'].unique())
print(thrs)

for rt60 in df_results_thr['rt60'].unique():

    for method_id in unique_method_ids:

        tpr_list = []
        fpr_list = []
        
        for t, thr in enumerate(thrs):

            tpr_list_ = []
            fpr_list_ = []
            
            df_ = df_results_thr.loc[
                    (df_results_thr['method_id'] == method_id)
                    & (df_results_thr['rt60'] == rt60)
                ]
            frame_ids = df_['frame_id'].unique()
                        
            for frame_id in unique_frame_ids:

                df__ = df_.loc[(df_['frame_id'] == frame_id)]
                    
                tpr = np.mean(df__[df__['thresholds_ang_spec'] == thr]['tpr'].values)
                fpr = np.mean(df__[df__['thresholds_ang_spec'] == thr]['fpr'].values)
                peaks_id = df__[df__['thresholds_ang_spec'] == thr]['peaks_ids'].values
                
                # check that tpr and fpr are not nans
                if np.isnan(tpr) or np.isnan(fpr):
                    print(frame_id)
                    print(peaks_id)
                    print(tpr)
                    print(fpr)
                    print(method_id)
                    print(df__)
                    raise ValueError('TPR or FPR is nan')
                
                assert len(np.unique(tpr)) == len(np.unique(fpr)) == 1
                            
                tpr_list_.append(tpr)
                fpr_list_.append(fpr)
                
            
            tpr_list.append(np.average(tpr_list_))
            fpr_list.append(np.average(fpr_list_))
            
        # plot ROC curve
        print(fpr_list)
        print(tpr_list)
        plt.plot(fpr_list, tpr_list, marker='o', label=method_id)
        
    # plt.legend()
    plt.show()

[0.   0.05 0.1  0.2  0.5  0.6  0.8  0.9 ]
nSrc-3_doas-[47  9 14]_type-speech-duration-1.0-snr-20_noise-awgn_reverb-0.123_mc-2
[]
nan
nan
music_s-1_freqs-[200, 4000]_gp-steerer_nObs-8_seed-666_norm-True
Empty DataFrame
Columns: [true_positives, false_positives, false_negatives, precision, recall, tpr, fpr, f1_score, accuracy, mean_error, frame_id, method_id, metrics_angular_threshold, thresholds_ang_spec, exp_name, time, record_id, num_srcs, src_ids, doas_est_idx, doas_ref_idx, doas_ref_az, doas_est_az, doas_ref_el, doas_est_el, errors, speech_files, target_doa, n_sources, duration, snr, noise_type, rt60, mc_seed, loc_method, freq_min, freq_max, sv_method, nObs, seed, sv_normalization, loc_method_simple, sv_model, loc_algo, peaks_locations, peaks_ids, n_peaks]
Index: []

[0 rows x 47 columns]


/Users/chutlhu/miniconda3/envs/shamans/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/chutlhu/miniconda3/envs/shamans/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: TPR or FPR is nan

<Figure size 800x600 with 0 Axes>

In [ ]:
tpr_list

In [ ]:
tpr_list_